
01_data_preparation


In [1]:
import pandas as pd
file_path = "Enterprise_Sales_Pipeline_Challenge_35.xlsx"
fact_deals = pd.read_excel(file_path, sheet_name="FactDeals")
fact_activities = pd.read_excel(file_path, sheet_name="FactActivities")
dim_company = pd.read_excel(file_path, sheet_name="DimCompany")
dim_salesrep = pd.read_excel(file_path, sheet_name="DimSalesRep")
dim_date = pd.read_excel(file_path, sheet_name="DimDate")

In [2]:
deals = fact_deals.merge(
    dim_company,
    on="CompanyID",
    how="left")

In [3]:
deals = deals.merge(
    dim_salesrep,
    on="RepID",
    how="left")

In [4]:
deals = deals.merge(
    dim_date.add_prefix("Created_"),
    left_on="CreatedDateKey",
    right_on="Created_DateKey",
    how="left")

In [5]:
deals = deals.merge(
    dim_date.add_prefix("LastActivity_"),
    left_on="LastActivityDateKey",
    right_on="LastActivity_DateKey",
    how="left")

In [6]:
activity_summary = fact_activities.groupby("DealID").agg(
    TotalActivities=("ActivityID", "count"),
    LastActivityDateKey=("ActivityDateKey", "max")).reset_index()

In [7]:
deals = deals.merge(
    activity_summary,
    on="DealID",
    how="left")

In [8]:
deals.to_csv(
    "processed_enterprise_sales_pipeline.csv",
    index=False)

In [9]:
df = pd.read_csv("processed_enterprise_sales_pipeline.csv")
print(df)

       DealID  CompanyID  RepID  CreatedDateKey  LastActivityDateKey_x  \
0           1        654     12        20230827               20230831   
1           2        497     51        20230403               20230406   
2           3        831      4        20230807               20240918   
3           4        435     34        20230917               20241025   
4           5        794     10        20221119               20230507   
...       ...        ...    ...             ...                    ...   
11745   11746        430     38        20250327               20250615   
11746   11747         64     30        20250516               20250703   
11747   11748        376      4        20250404               20250702   
11748   11749        818      9        20250715               20251204   
11749   11750        404     26        20250826               20260131   

         StageName  StageOrder Status  BaseWinProbability  DealValueEUR  ...  \
0          Lead In           1 

In [10]:
print(df.shape)
print(df["DealID"].nunique())
print(df.isnull().sum().sort_values(ascending=False).head(10))

(11750, 41)
11750
LeaveYear            9057
DealID                  0
Role                    0
JoinYear                0
Created_Date            0
Created_DateKey         0
Created_Year            0
Created_Quarter         0
Created_Month           0
Created_MonthName       0
dtype: int64



02_kpi_engineering


In [11]:
# what is the percent of won revenue in the deals?
total_pipeline = df["DealValueEUR"].sum()
won_revenue = df.loc[df["Status"] == "Won", "DealValueEUR"].sum()
lost_revenue = df.loc[df["Status"] == "Lost", "DealValueEUR"].sum()
win_rate = (df["Status"] == "Won").mean()
win_percentage = str(round((win_rate * 100),2)) + "%"
print(f"Total Pipeline: €{total_pipeline:,}")
print(f"Won Revenue: €{won_revenue:,}")
print(f"Lost Revenue: €{lost_revenue:,}")
print(f"Win%:", win_percentage)

Total Pipeline: €1,022,703,320
Won Revenue: €240,815,432
Lost Revenue: €175,850,684
Win%: 25.52%


In [12]:
# what is the weighted pipeline value?
df["WeightedValue"] = df["DealValueEUR"] * df["BaseWinProbability"]
weighted_pipeline = df["WeightedValue"].sum()
print(f"Weighted Pipeline Value: €{int(weighted_pipeline):,}")

Weighted Pipeline Value: €429,079,715


In [13]:
#what is the average deal cycle 
df["DealCycleDays"] = (pd.to_datetime(df["LastActivity_Date"]) - pd.to_datetime(df["Created_Date"])).dt.days
avg_cycle = df["DealCycleDays"].mean()
print(f" Average Deal Cycle: {round(avg_cycle)} Days")

 Average Deal Cycle: 131 Days


In [14]:
#what is the average deal size
avg_deal_size = round(df["DealValueEUR"].mean())
sales_velocity = round((df.shape[0] * win_rate * avg_deal_size) / avg_cycle)
print(f"Average Deal Size: €{avg_deal_size:,}")
print(f"Sales Velocity: {sales_velocity:,}")

Average Deal Size: €87,039
Sales Velocity: 1,996,549


03_Stastical_Testing & Probablility

In [17]:
#what is the Bucket Base Win Probability
df["ProbBucket"] = pd.cut(
    df["BaseWinProbability"],
    bins=[0, 0.25, 0.5, 0.75, 1.0],
    labels=["0-25%", "26-50%", "51-75%", "76-100%"],include_lowest=True)
calibration = df.groupby("ProbBucket")["Status"].agg(
    ActualWinRate=lambda x: (x == "Won").mean(), DealCount="count").sort_index()
print("Probability Calibration Report")
for bucket, row in calibration.iterrows():
    actual = f"{row['ActualWinRate']:.0%}"
    count = f"({int(row['DealCount'])} deals)"
    print(f"Predicted {bucket:8} | Actual Win Rate: {actual:>4} | {count}")

Probability Calibration Report
Predicted 0-25%    | Actual Win Rate:  25% | (4853 deals)
Predicted 26-50%   | Actual Win Rate:  27% | (2482 deals)
Predicted 51-75%   | Actual Win Rate:  26% | (2294 deals)
Predicted 76-100%  | Actual Win Rate:  25% | (2121 deals)


In [18]:
import numpy as np
df["WonBinary"] = (df["Status"] == "Won").astype(int)
correlation = np.corrcoef(df["BaseWinProbability"], df["WonBinary"])[0,1]
print("Correlation between Probability and Actual Outcome:", correlation)

Correlation between Probability and Actual Outcome: 0.02313577565674174


Probability assignments across the pipeline show no meaningful relationship with actual deal outcomes. Win rates remain statistically flat (~25–27%) across all probability bands, indicating a lack of predictive validity in the current forecasting methodology.

DEAL CYCLE STATISTICAL TESTING (Behavior Efficiency)

In [25]:
#Comparing Won vs Lost Cycle
from scipy.stats import ttest_ind
won_cycle = df[df["Status"]=="Won"]["DealCycleDays"]
lost_cycle = df[df["Status"]=="Lost"]["DealCycleDays"]
ttest_ind(won_cycle, lost_cycle)

Ttest_indResult(statistic=2.9154318250818037, pvalue=0.003567814132803089)

Interpretation:

p < 0.05 → cycle length significantly impacts outcome

p > 0.05 → cycle not statistically different
with the P < 0.05, THis means that the cycle length significantly impacts outcome.


04_Analysis


REP PERFORMANCE Analysis (People Efficiency)

In [16]:
# what is the rep efficiency index
df["WonRevenue"] = df.apply(lambda x: x["DealValueEUR"] if x["Status"] == "Won" else 0, axis=1)
rep_perf = df.groupby("RepID").agg(
    Revenue=("WonRevenue", "sum"),
    WinRate=("Status", lambda x: (x == "Won").mean()),
    AvgCycle=("DealCycleDays", "mean"))
rep_perf["EfficiencyIndex"] = (rep_perf["Revenue"] * rep_perf["WinRate"]) / rep_perf["AvgCycle"]
rep_perf = rep_perf.sort_values(by="EfficiencyIndex", ascending=False)
print("Sales Rep Efficiency Index")
for rep, row in rep_perf.iterrows():
    rev = f"€{int(row['Revenue']):,}"
    win = f"{row['WinRate']:.0%}"
    eff = f"{int(row['EfficiencyIndex']):,}"
    print(f"Rep {rep:2} | Efficiency:{eff:>7} | Rev:{rev:>10} | Win:{win:>4}")

Sales Rep Efficiency Index
Rep 30 | Efficiency: 31,217 | Rev:€16,735,281 | Win: 26%
Rep 20 | Efficiency: 18,618 | Rev:€9,995,607 | Win: 21%
Rep 38 | Efficiency: 17,929 | Rev:€8,076,424 | Win: 28%
Rep 26 | Efficiency: 17,832 | Rev:€9,089,927 | Win: 21%
Rep 17 | Efficiency: 17,523 | Rev:€7,475,963 | Win: 33%
Rep  5 | Efficiency: 15,804 | Rev:€7,400,601 | Win: 31%
Rep  2 | Efficiency: 14,941 | Rev:€8,916,374 | Win: 25%
Rep 23 | Efficiency: 14,554 | Rev:€6,849,685 | Win: 30%
Rep 21 | Efficiency: 13,939 | Rev:€7,459,756 | Win: 27%
Rep  4 | Efficiency: 13,253 | Rev:€7,244,374 | Win: 26%
Rep 34 | Efficiency: 13,115 | Rev:€6,837,677 | Win: 27%
Rep 12 | Efficiency: 12,042 | Rev:€5,577,805 | Win: 30%
Rep 52 | Efficiency: 11,653 | Rev:€3,867,814 | Win: 35%
Rep 55 | Efficiency: 11,598 | Rev:€5,415,899 | Win: 24%
Rep  7 | Efficiency: 11,280 | Rev:€5,739,571 | Win: 31%
Rep 43 | Efficiency: 10,909 | Rev:€6,422,410 | Win: 18%
Rep 37 | Efficiency: 10,763 | Rev:€5,629,527 | Win: 27%
Rep  3 | Efficiency:

Funnel Leakage Analysis:
where are we losing our data?

In [19]:
# what is the stage entry volume
stage_counts = df.groupby("StageName").agg(Deals=("DealID", "count"),TotalValue=("DealValueEUR", "sum")).sort_values("Deals", ascending=False)
stage_counts_display = stage_counts.copy()
stage_counts_display["TotalValue"] = stage_counts_display["TotalValue"].apply(lambda x: f"€{int(x):,}")
print("Stage Entry Volume Report")
stage_counts_display

Stage Entry Volume Report


,Deals,TotalValue
StageName,,
Lead In,3125,"€289,251,695"
Qualified,2588,"€230,110,426"
Proposal,2414,"€204,344,781"
Negotiation,2021,"€173,376,271"
Closing,846,"€42,790,539"
Contracting,756,"€82,829,608"


In [20]:
# what is the stage win rate?
stage_stat = df.groupby("StageName")["Status"].agg(
    WinRate=lambda x: (x == "Won").mean(),
    DealCount="count").sort_values("WinRate", ascending=False)
stage_stat["WinRate"] = stage_stat["WinRate"].apply(lambda x: f"{x:.0%}")
print("Stage Win Rate")
stage_stat

Stage Win Rate


,WinRate,DealCount
StageName,,
Closing,32%,846
Negotiation,26%,2021
Proposal,26%,2414
Qualified,26%,2588
Lead In,24%,3125
Contracting,21%,756


In [21]:
#what is the revenue leakage per stage?
stage_leakage = df[df["Status"] == "Lost"].groupby("StageName")["DealValueEUR"].sum().sort_values(ascending=False)
leakage_report = stage_leakage.to_frame(name="RevenueLost")
leakage_report["RevenueLost"] = leakage_report["RevenueLost"].apply(lambda x: f"€{int(x):,}")
print("Revenue Leakage by Stage")
leakage_report

Revenue Leakage by Stage


,RevenueLost
StageName,
Lead In,"€51,165,124"
Qualified,"€40,339,526"
Proposal,"€33,481,894"
Negotiation,"€27,787,548"
Contracting,"€15,325,480"
Closing,"€7,751,112"


In [22]:
#What is Stage conversion by order?
stage_flow = df.groupby("StageOrder").agg(
    Stage=("StageName", "first"),
    Deals=("DealID", "count"),
    Wins=("Status", lambda x: (x == "Won").sum())).sort_index()
stage_flow["ConversionRate"] = stage_flow["Wins"] / stage_flow["Deals"]
stage_flow_display = stage_flow.copy()
stage_flow_display["ConversionRate"] = stage_flow_display["ConversionRate"].apply(lambda x: f"{x:.0%}")
print("Pipeline Flow & Conversion Analysis")
stage_flow_display

Pipeline Flow & Conversion Analysis


,Stage,Deals,Wins,ConversionRate
StageOrder,,,,
1,Lead In,3125,737,24%
2,Qualified,2588,670,26%
3,Proposal,2414,632,26%
4,Negotiation,2021,531,26%
5,Closing,1602,429,27%


In [15]:
#what is the deal cycle impact on win rate
df["CycleBucket"] = pd.cut(
    df["DealCycleDays"],
    bins=[0, 30, 60, 90, 120, 365],
    labels=["0-30", "31-60", "61-90", "91-120", "120+"])
cycle_analysis = df.groupby("CycleBucket")["Status"].agg(
    win_rate=lambda x: (x == "Won").mean(),
    deal_count="count")
cycle_analysis = cycle_analysis.sort_values(by="win_rate", ascending=False)
print("Deal Cycle Impact on Win Rate")
for bucket, row in cycle_analysis.iterrows():
    print(f"Cycle {bucket:7} | Win Rate: {row['win_rate']:>4.2%} | (Deals: {int(row['deal_count'])})")

Deal Cycle Impact on Win Rate
Cycle 0-30    | Win Rate: 27.19% | (Deals: 2012)
Cycle 91-120  | Win Rate: 24.84% | (Deals: 1409)
Cycle 31-60   | Win Rate: 24.38% | (Deals: 1842)
Cycle 61-90   | Win Rate: 24.00% | (Deals: 1675)
Cycle 120+    | Win Rate: 23.34% | (Deals: 3411)


04_Summary & Recommendation

This analysis evaluates 11,750 enterprise sales opportunities to assess forecasting reliability, funnel efficiency, deal cycle impact, and sales performance distribution across the organization.
The objective was to determine whether current pipeline metrics accurately reflect revenue reality and to identify structural drivers affecting commercial performance.
1. Revenue & Win Rate Overview:
Overall win rate: 27%,
Total pipeline includes 11,750 enterprise opportunities,
Revenue performance shows moderate conversion efficiency but limited predictive clarity  while headline win rate appears stable, deeper analysis reveals structural weaknesses in forecasting logic and stage progression behavior.
2. Forecasting Reliability: Probability Scores Lack Predictive Power
A probability calibration audit revealed:
Predicted Probability Range	Actual Win Rate
0–25%: 25%, 26–50%:	27%, 51–75%:	26%, 76–100%:	25%
- Insight:
Assigned probability scores show no meaningful correlation with actual deal outcomes.
High-probability deals close at nearly the same rate as low-probability deals.
- Implication:
Weighted pipeline revenue is materially overstated,
Forecast confidence is artificially inflated,
Revenue planning decisions are exposed to miscalibration risk.
* This is a structural forecasting governance issue not a performance issue.
3. Funnel Leakage & Stage Efficiency:
Stage transition analysis identifies uneven conversion patterns across the pipeline.
- Findings:
Significant revenue attrition occurs in mid-funnel stages,
Stage progression does not consistently improve win likelihood,
Certain stages act as accumulation points rather than qualification gates.
- Implication:
Pipeline inefficiencies are likely driven by: Qualification breakdown
,Inconsistent stage definitions,
Negotiation stagnation, 
* Optimizing stage discipline represents an immediate revenue opportunity.
4. Deal Cycle Impact on Performance:
Statistical testing indicates a relationship between deal duration and closing success.
Longer sales cycles are associated with:
Lower win probability,
Increased probability stagnation,
Revenue timing uncertainty,
- Implication:
Cycle acceleration strategies could materially improve:
Revenue predictability,
Sales velocity,
Forecast reliability.
* Cycle duration should be treated as a performance KPI not just a reporting metric.
5. Sales Representative Performance Intelligence:
Revenue distribution analysis shows performance concentration across representatives.
A minority of reps generate a disproportionate share of revenue. Efficiency variation across reps is significant.
Performance volatility introduces execution risk
- Implication:
There is opportunity to:Standardize best-practice behaviors,
Coach bottom-quartile performers,
Reduce concentration dependency risk.
6. Financial Opportunity Outlook:
If the organization were to achieve +5 percentage point improvement in win rate, Improved probability calibration,Reduced average sales cycle duration,
The revenue uplift potential is substantial.
Even modest improvements in conversion efficiency would generate material incremental revenue without increasing pipeline volume.
* Overall Strategic Assessment:The sales organization demonstrates stable headline performance but underlying structural inefficiencies in:
- Forecast calibration
- Stage governance
- Probability scoring logic
- Sales cycle management
- Performance distribution consistency
Addressing these areas represents a high-leverage revenue optimization opportunity.
- Recommended Focus Areas
- Implement probability calibration framework based on historical outcomes
- Redefine stage exit criteria to reduce mid-funnel leakage
- Introduce cycle duration guardrails and escalation triggers
- Deploy performance benchmarking dashboard for rep-level efficiency
- Align forecast weighting methodology with empirical win behavior
* Final Executive Conclusion:The pipeline does not currently function as a predictive instrument, it functions as a volume tracker.With structural refinement, it can become a strategic revenue intelligence engine.